In [1]:
import sys
sys.path.append('../')

In [2]:
from Datasets.CharTokenizer import CharTokenizer
from Datasets.SubwordTokenizer import SubwordTokenizer
from Datasets.ThDatasetVISTEC import ThDatasetVISTEC
# import torch

In [3]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
from util import *

# Alphabetization
* Standardize word form
```
    Word = [Initial Consonant] + [Vowel] + [Tone] + [Final Consonant]
```
* Unitize vowel

Note:
* No cases by default where
    * Tone > above/below vowels

In [5]:
words = []
with open("../Data/Dict/words_th.txt", encoding="utf-8") as fin:
    for line in fin:
        words.append(line.strip())

In [6]:
"Words", len(words)

('Words', 62056)

In [7]:


testcases = [
    ("อะ", ['อ', 'ะ']),
    ("อัก", ['อ', 'ั', 'ก']),
    ("อรร", ['อ', '<อรร>']),
    ("อรรก", ['อ', '<อรร>', 'ก']),
    ("อิก", ['อ', 'ิ', 'ก']),
    ("อึก", ['อ', 'ึ', 'ก']),
    ("อุก", ['อ', 'ุ', 'ก']),
    ("อก", ['อ', 'ก']),
    ("อ็อก", ['อ', '<อ็อx>', 'ก']),
    ("ออก", ['อ', 'อ', 'ก']),
    ("อัวะ", ['อ', '<อัวะ>']),
    ("อ็วก", ['อ', '<อ็วx>', 'ก']),
    ("อาก", ['อ', 'า', 'ก']),
    ("อีก", ['อ', 'ี', 'ก']),
    ("อืก", ['อ', 'ื', 'ก']),
    ("อูก", ['อ', 'ู', 'ก']),
    ("อร", ['อ', 'ร']),
    ("อัว", ['อ', '<อัว>']),
    ("อวก", ['อ', 'ว', 'ก']),
    ("เอา", ['อ', '<เอา>']),
    ("เอาะ", ['อ', '<เอาะ>']),
    ("เกราะ", ['ก', 'ร', '<เอาะ>']),
    ("เกลาะ", ['ก', 'ล', '<เอาะ>']),
    ("เกวาะ", ['ก', 'ว', '<เอาะ>']),
    ("เกวาะ", ['ก', 'ว', '<เอาะ>']),
    ("เหยาะ", ['ห', 'ย', '<เอาะ>']),
    ("เอือะ", ['อ', '<เอือะ>']),
    ("เออะ", ['อ', '<เออะ>']),
    ("เอิก", ['อ', '<เอิx>', 'ก']),
    ("โอก", ['อ', 'โ', 'ก']),
    ("เอะ", ['อ', '<เอะ>']),
    ("เอ็ก", ['อ', '<เอ็x>', 'ก']),
    ("เอก", ['อ', 'เ', 'ก']),
    ("แอะ", ['อ', '<แอะ>']),
    ("แอ็ก", ['อ', '<แอ็x>', 'ก']),
    ("แอก", ['อ', 'แ', 'ก']),
    ("โอะ", ['อ', '<โอะ>']),
    ("เอียก", ['อ', '<เอีย>', 'ก']),
    ("เอือก", ['อ', '<เอือ>', 'ก']),
    ("เออ", ['อ', '<เออ>']),
    ("เอย", ['อ', '<เอย>']),
    ("เออก", ['อ', '<เออ>', 'ก']),
    ("อ่ะ", ['อ', 'ะ', '่']),
    ("อั่ก", ['อ', 'ั', '่', 'ก']),
    ("อรร", ['อ', '<อรร>']),
    ("อรรก", ['อ', '<อรร>', 'ก']),
    ("อิ่ก", ['อ', 'ิ', '่', 'ก']),
    ("อึ่ก", ['อ', 'ึ', '่', 'ก']),
    ("อุ่ก", ['อ', 'ุ', '่', 'ก']),
    ("อก", ['อ', 'ก']),
    ("อ็อก", ['อ', '<อ็อx>', 'ก']),
    ("ออก", ['อ', 'อ', 'ก']),
    ("อั่วะ", ['อ', '<อัวะ>', '่']),
    ("อ็วก", ['อ', '<อ็วx>', 'ก']),
    ("อ่าก", ['อ', 'า', '่', 'ก']),
    ("อี่ก", ['อ', 'ี', '่', 'ก']),
    ("อื่ก", ['อ', 'ื', '่', 'ก']),
    ("อู่ก", ['อ', 'ู', '่', 'ก']),
    ("อร", ['อ', 'ร']),
    ("อั่ว", ['อ', 'ั', '่', 'ว']),
    ("เอ่า", ['อ', '<เอา>', '่']),
    ("เอ้าะ", ['อ', '<เอาะ>', '้']),
    ("เกร้าะ", ['ก', 'ร', '<เอาะ>', '้']),
    ("เกล้าะ", ['ก', 'ล', '<เอาะ>', '้']),
    ("เกว้าะ", ['ก', 'ว', '<เอาะ>', '้']),
    ("เหยาะ", ['ห', 'ย', '<เอาะ>']),
    ("เอื๊อะ", ['อ', '<เอือะ>', '๊']),
    ("เอ๊อะ", ['อ', '<เออะ>', '๊']),
    ("เอิ๊ก", ['อ', '<เอิx>', '๊', 'ก']),
    ("โอ๊ก", ['อ', 'โ', '๊', 'ก']),
    ("เอ๊ะ", ['อ', '<เอะ>', '๊']),
    ("เอ็ก", ['อ', '<เอ็x>', 'ก']),
    ("เอ๊ก", ['อ', 'เ', '๊', 'ก']),
    ("แอะ", ['อ', '<แอะ>']),
    ("แอ็ก", ['อ', '<แอ็x>', 'ก']),
    ("แอ๊ก", ['อ', 'แ', '๊', 'ก']),
    ("โอ๊ะ", ['อ', '<โอะ>', '๊']),
    ("เอี๊ยก", ['อ', '<เอีย>', '๊', 'ก']),
    ("เอื๊อก", ['อ', '<เอือ>', '๊', 'ก']),
    ("เอ้อ", ['อ', '<เออ>', '้']),
    ("เอ้ย", ['อ', '<เอย>', '้']),
    ("เอ้อก", ['อ', '<เออ>', '้', 'ก']),
    ("เอา", ['อ', '<เอา>']),
    ("เอ้า", ['อ', '<เอา>', "้"]),
    
    ("เอิน", ['อ', '<เอิx>', "น"]),
    ("เอิ้น", ['อ', '<เอิx>', "้", "น"]),
    
    # ("อ๊วก", ['อ', '๊', 'ว', 'ก']),
    # ("อ้อ", ['อ', "อ", '้']),
    
]

for w, expected in testcases:
    tokens = alphabetize(w)
    if tokens!=expected:
        print("Original", w)
        print("Expected", expected)
        print("Actual", tokens)
        print()
    assert(tokens==expected)

print("DONE")
# for d in maindataset.datasets["test"]:
#     sent = alphabetize(d["sent"])
# #     print("Original", d["sent"])
# #     print()
# #     print("Alphabetized", sent)
# #     break

DONE


In [14]:
import random
cc = 0
random.shuffle(words)

for w in words:
    if cc > 10:
        break
    
    if random.random() > 0.5:
        tokens = alphabetize(w)
        print("Original", w)
        print("Actual", tokens)
        print()
        cc += 1


Original ไชโป๊
Actual ['ช', 'ไ', 'ป', 'โ', '๊']

Original หนึ่ง
Actual ['ห', 'น', 'ึ', '่', 'ง']

Original พิมล
Actual ['พ', 'ิ', 'ม', 'ล']

Original เก่งแต่ปาก
Actual ['ก', 'เ', '่', 'ง', 'ต', 'แ', '่', 'ป', 'า', 'ก']

Original ทวิบาท
Actual ['ท', 'ว', 'ิ', 'บ', 'า', 'ท']

Original บริกรรม
Actual ['บ', 'ร', 'ิ', 'ก', '<อรร>', 'ม']

Original ราว ๆ
Actual ['ร', 'า', 'ว', ' ', 'ๆ']

Original ตรีเสมหผล
Actual ['ต', 'ร', 'ี', 'ส', 'เ', 'ม', 'ห', 'ผ', 'ล']

Original ผู้กำกับการ
Actual ['ผ', 'ู', '้', 'ก', 'ำ', 'ก', 'ั', 'บ', 'ก', 'า', 'ร']

Original เคมี
Actual ['ค', 'เ', 'ม', 'ี']

Original สัญจาระ
Actual ['ส', 'ั', 'ญ', 'จ', 'า', 'ร', 'ะ']



In [15]:
def save_file(filename, data):
    with open(filename, "w", encoding="utf-8") as fout:
        for d in data:
            fout.write(d)
            fout.write("\n")

In [16]:
# maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-sample", name="VISTEC-sample")
maindataset = ThDatasetVISTEC("../Data/VISTEC-TP-TH-2021", name="VISTEC")

In [17]:
from collections import defaultdict
uniqueTokens = set()
nTokens = defaultdict(int)

charSents = []
for d in maindataset.datasets["train"]:
    sent = d["sent"]
    char = list(sent)
    charSents.append(" ".join(char))
    uniqueTokens.update(char)
    for c in char:
        nTokens[c] += 1
#     break


N = 0
for k in nTokens:
    N += nTokens[k]
print("#UniqueTokens", len(uniqueTokens))
print("#Tokens", N)
# save_file("../fasttext_char.txt", charSents)

#UniqueTokens 164
#Tokens 10862818


In [18]:
from tqdm import tqdm
uniqueTokens = set()
nTokens = defaultdict(int)

alphSents = []

with tqdm(total=40000) as pbar:
    for d in maindataset.datasets["train"]:
        sent = d["sent"]
        char = alphabetize(sent)
        alphSents.append(" ".join(char))
        uniqueTokens.update(char)
        for c in char:
            nTokens[c] += 1
        
        pbar.update(1)


N = 0
for k in nTokens:
    N += nTokens[k]
print("#UniqueTokens", len(uniqueTokens))
print("#Tokens", N)


# save_file("../fasttext_alph.txt", alphSents)

 98%|██████████████████████████████████▏| 39000/40000 [00:31<00:00, 1224.30it/s]

#UniqueTokens 185
#Tokens 10278787


In [20]:
print(f"Diff: {(10862818-10278782)*100/10862818:.2f}%")

Diff: 5.38%


## Train Fasttext

In [20]:
import fasttext

In [21]:
# wvChar = fasttext.train_unsupervised('../fasttext_char.txt', epoch=200, ws=3)

In [22]:
# wvChar.save_model('../Models/wvchar.bin')

In [23]:
# wvAlph = fasttext.train_unsupervised('../fasttext_alph.txt', epoch=200, ws=3)

In [24]:
# wvAlph.save_model('../Models/wvalph.bin')

# Intrinsic Evaluation

In [25]:
sys.path

['/home/imtk/Desktop/AlphabetizedThai/Notebooks',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/imtk/.local/lib/python3.8/site-packages',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/home/imtk/.local/lib/python3.8/site-packages/IPython/extensions',
 '/home/imtk/.ipython',
 '../']

In [26]:
import sys
sys.path.append('../Libs/word-embeddings-benchmarks/')

In [27]:
import logging, sys
import scipy.stats
from six import iteritems
from web.datasets.similarity import fetch_MEN, fetch_WS353, fetch_SimLex999
from web.datasets.similarity import fetch_TWS65, fetch_thai_wordsim353, fetch_thai_semeval2017_task2, fetch_thai_simlex999
from web.embeddings import load_embedding
# from web.evaluate import evaluate_similarity

In [28]:
import sklearn

In [29]:
import numpy as np
from scipy import spatial
    
def evaluate_similarity(wv, X, y, preprocess=None):
    
    missing_words, found_words, oov_vecs_created, index = 0, 0, 0, 0
    word_pair_oov_indices = []
    info_oov_words = {}
    info_created_words = {}

    ## For all words in the datasets, check if the are OOV? 
    ## Indices of word-pairs with a OOV word are stored in word_pair_oov_indices
    
    nwords = 0
    for query in X:
        for query_word in query:
            found_words += 1
            nwords += 1
        index += 1

    # print(f"Missing Word: {missing_words} words ({missing_words*100/nwords:.2f}%)")
    

    # The original code; for all OOV; it will be replaced by average vector
    # mean_vector = np.mean(w.vectors, axis=0, keepdims=True)
    # A = np.vstack(w.get(word, mean_vector) for word in X[:, 0])
    # B = np.vstack(w.get(word, mean_vector) for word in X[:, 1])
    # scores = np.array([v1.dot(v2.T)/(np.linalg.norm(v1)*np.linalg.norm(v2)) for v1, v2 in zip(A, B)])
    
    scores = []
    for w in X:
        vecA = wv.get_vector(w[0])
        vecB = wv.get_vector(w[1])
        s = 1 - spatial.distance.cosine(vecA, vecB)
        scores.append(s)
        
#     A = np.vstack(w[preprocess(word)] for word in )
#     B = np.vstack(w[preprocess(word)] for word in X[:, 1])
#     scores = np.array([v1.dot(v2.T)/(np.linalg.norm(v1)*np.linalg.norm(v2)) for v1, v2 in zip(A, B)])


    # wohlg: original version only returned Spearman 
    # wohlg: we added Pearson and other information 
    result = {
        'spearmanr': scipy.stats.spearmanr(scores, y).correlation,
        'pearsonr':  scipy.stats.pearsonr(scores, y)[0],
        'num_oov_word_pairs': len(word_pair_oov_indices),
        'num_found_words': found_words,
        'num_missing_words': missing_words,
        "num_word_pairs": nwords
    }

    return result


In [30]:
import fasttext


In [31]:
tasks = {
    "TH-WS353": fetch_thai_wordsim353(),
    "TH-SemEval2017T2": fetch_thai_semeval2017_task2(),
    "TH-SimLex999": fetch_thai_simlex999(),
    "TWS65": fetch_TWS65()
}

# Print sample data
for name, data in iteritems(tasks):
    print("Sample data from {}: pair \"{}\" and \"{}\" is assigned score {}".format(name, data.X[0][0], data.X[0][1], data.y[0]))

def eval_word_sim(wv, verbose=True):
    # Calculate results using helper function for the various word similarity datasets
    results = {}
    for name, data in iteritems(tasks):
        result = evaluate_similarity(wv, data.X, data.y)

    #     hm = scipy.stats.hmean([result['spearmanr'], result['pearsonr']])
        perc_oov_words = 100 * (result['num_missing_words'] / (result['num_found_words'] + float(result['num_missing_words'])))

        # Spearman: evaluate a monotonic relationship between two variables based on the ranked values for each variable rather than the raw data.
        # Pearson : measures the linear correlation between two variables X and Y
        if verbose:
            print(f"Dataset {name}: Spearman: {result['spearmanr']:4.3f}")
        results[name] = result['spearmanr']
    return results


Sample data from TH-WS353: pair "ความรัก" and "เพศ" is assigned score 12.2
Sample data from TH-SemEval2017T2: pair "จูล" and "ยานอวกาศ" is assigned score 1.9
Sample data from TH-SimLex999: pair "เก่า" and "ใหม่" is assigned score 4.38
Sample data from TWS65: pair "แก้ว" and "ข้ารับใช้" is assigned score 0.116


## Target on Fasttext on Word-level

W/o OOV
* Dataset TH-WS353: Spearman: 0.182
* Dataset TH-SemEval2017T2: Spearman: 0.175
* Dataset TH-SimLex999: Spearman: 0.201
* Dataset TWS65: Spearman: 0.203

With OOV
* Dataset TH-WS353: Spearman: 0.347
* Dataset TH-SemEval2017T2: Spearman: 0.371
* Dataset TH-SimLex999: Spearman: 0.410
* Dataset TWS65: Spearman: 0.252

Cite: [Word Similarity Datasets for Thai: Construction and Evaluation](https://ieeexplore.ieee.org/document/8851127)

In [32]:
class WordVector:
    def __init__(self, path, wv=None):
        if wv is None:
            wv = fasttext.load_model(path)
            
        self.wv = wv
        
    def get_vector(self, word):
        wv = self.wv
    #     return wv[word]

    #     print(" ".join(list(word)))
    #     return wv.get_sentence_vector(" ".join(list(word)))

        vec = wv[word[0]]
        for c in word[1:]:
            vec += wv[c]
        vec /= len(word)
        return vec

In [33]:
nexp = 10
epoch = 200

In [34]:
wv = WordVector("../Models/wvchar.bin")
wordsim = eval_word_sim(wv)

Dataset TH-WS353: Spearman: 0.208
Dataset TH-SemEval2017T2: Spearman: 0.251
Dataset TH-SimLex999: Spearman: 0.270
Dataset TWS65: Spearman: -0.136


In [35]:
from collections import defaultdict

results = defaultdict(list)
for i in range(nexp):
    model = fasttext.train_unsupervised('../fasttext_char.txt', epoch=epoch)
    wv = WordVector('', model)
    
    wordsim = eval_word_sim(wv, verbose=False)
    for k in wordsim:
        results[k].append(wordsim[k])

for name in results:
    print(f"Dataset {name}: Spearman: {np.mean(results[name]):.3f} {np.std(results[name]):.3f}")

Read 10M words
Number of words:  162
Number of labels: 0
Progress: 100.0% words/sec/thread:  543044 lr:  0.000000 avg.loss:  0.798446 ETA:   0h 0m 0s 33.8% words/sec/thread:  549039 lr:  0.033115 avg.loss:  2.019515 ETA:   0h 3m48s
Read 10M words
Number of words:  162
Number of labels: 0
Progress: 100.0% words/sec/thread:  543250 lr:  0.000000 avg.loss:  0.803694 ETA:   0h 0m 0s 84.7% words/sec/thread:  543739 lr:  0.007653 avg.loss:  0.923118 ETA:   0h 0m53s
Read 10M words
Number of words:  162
Number of labels: 0
Progress: 100.0% words/sec/thread:  546325 lr:  0.000000 avg.loss:  0.807703 ETA:   0h 0m 0s 35.5% words/sec/thread:  548511 lr:  0.032244 avg.loss:  1.948769 ETA:   0h 3m42s 50.3% words/sec/thread:  547950 lr:  0.024851 avg.loss:  1.438410 ETA:   0h 2m51s100.0% words/sec/thread:  546325 lr: -0.000000 avg.loss:  0.807703 ETA:   0h 0m 0s
Read 10M words
Number of words:  162
Number of labels: 0
Progress: 100.0% words/sec/thread:  536577 lr:  0.000000 avg.loss:  0.813014 ETA:  

Dataset TH-WS353: Spearman: 0.215 0.004
Dataset TH-SemEval2017T2: Spearman: 0.246 0.003
Dataset TH-SimLex999: Spearman: 0.268 0.002
Dataset TWS65: Spearman: -0.133 0.006


Progress: 100.0% words/sec/thread:  538432 lr:  0.000000 avg.loss:  0.802294 ETA:   0h 0m 0s


In [36]:
class WordVectorAlph:
    def __init__(self, path, wv=None):
        if wv is None:
            wv = fasttext.load_model(path)

        self.wv = wv
        
    def get_vector(self, word):
        word = alphabetize(word)
        wv = self.wv

        vec = wv[word[0]]
        for c in word[1:]:
            vec += wv[c]
        vec /= len(word)
        return vec

In [37]:
wv = WordVectorAlph('../Models/wvalph.bin')
wordsim = eval_word_sim(wv)

Dataset TH-WS353: Spearman: 0.175
Dataset TH-SemEval2017T2: Spearman: 0.258
Dataset TH-SimLex999: Spearman: 0.246
Dataset TWS65: Spearman: -0.161


In [38]:
from collections import defaultdict

results = defaultdict(list)
for i in range(nexp):
    model = fasttext.train_unsupervised('../fasttext_char.txt', epoch=epoch)
    wv = WordVector('', model)
    
    wordsim = eval_word_sim(wv, verbose=False)
    for k in wordsim:
        results[k].append(wordsim[k])

for name in results:
    print(f"Dataset {name}: Spearman: {np.mean(results[name]):.3f} {np.std(results[name]):.3f}")

Read 10M words
Number of words:  162
Number of labels: 0
Progress: 100.0% words/sec/thread:  545740 lr:  0.000000 avg.loss:  0.784073 ETA:   0h 0m 0s 2.401615 ETA:   0h 4m58s
Read 10M words
Number of words:  162
Number of labels: 0
Progress: 100.0% words/sec/thread:  546639 lr:  0.000000 avg.loss:  0.806044 ETA:   0h 0m 0s 18.9% words/sec/thread:  549773 lr:  0.040541 avg.loss:  2.617630 ETA:   0h 4m38s 55.2% words/sec/thread:  548005 lr:  0.022385 avg.loss:  1.324049 ETA:   0h 2m34s 94.0% words/sec/thread:  546785 lr:  0.002998 avg.loss:  0.848507 ETA:   0h 0m20s
Read 10M words
Number of words:  162
Number of labels: 0
Progress: 100.0% words/sec/thread:  541220 lr:  0.000000 avg.loss:  0.747567 ETA:   0h 0m 0s 1.913655 ETA:   0h 3m42s 0.028336 avg.loss:  1.625025 ETA:   0h 3m17s 2m17s
Read 10M words
Number of words:  162
Number of labels: 0
Progress: 100.0% words/sec/thread:  543394 lr:  0.000000 avg.loss:  0.756959 ETA:   0h 0m 0s
Read 10M words
Number of words:  162
Number of labels

Dataset TH-WS353: Spearman: 0.216 0.004
Dataset TH-SemEval2017T2: Spearman: 0.247 0.002
Dataset TH-SimLex999: Spearman: 0.268 0.002
Dataset TWS65: Spearman: -0.134 0.006


## LSTM Model

In [39]:
# TODO:

In [40]:
"DONE"

'DONE'